In [1]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append(".")
# os.environ["MAX_JOBS"] = "100"

import torch
torch.set_printoptions(linewidth=200)
torch.cuda.is_available()

True

In [2]:
from src.vlstm_fw_v1.interface import vlstm_fw_torch
from src.vlstm_fw_v1.interface import vlstm_fw_cuda
from vlstm_formulations.vlstm_parallel import vlstm_parallel_fw_torch
from vlstm_formulations.vlstm_parallel_tiled import vlstm_parallel_tiled

# Commit: 59a91237f7e6733247aa5d2b3da8a66970e7bf54
# ptxas info    : 343 bytes gmem
# ptxas info    : Compiling entry function '_ZN5vlstm7kernels8vlstm_fwIfLi4ELi8ELi8EEEvPT_S3_S3_S3_iiii' for 'sm_89'
# ptxas info    : Function properties for _ZN5vlstm7kernels8vlstm_fwIfLi4ELi8ELi8EEEvPT_S3_S3_S3_iiii
#     24 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
# ptxas info    : Used 66 registers, 400 bytes cmem[0]
# ptxas info    : Compiling entry function '_ZN5vlstm7kernels8vlstm_fwI6__halfLi4ELi8ELi8EEEvPT_S4_S4_S4_iiii' for 'sm_89'
# ptxas info    : Function properties for _ZN5vlstm7kernels8vlstm_fwI6__halfLi4ELi8ELi8EEEvPT_S4_S4_S4_iiii
#     24 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
# ptxas info    : Used 72 registers, 400 bytes cmem[0]
# ptxas info    : Compiling entry function '_ZN5vlstm7kernels8vlstm_fwI13__nv_bfloat16Li4ELi8ELi8EEEvPT_S4_S4_S4_iiii' for 'sm_89'
# ptxas info    : Function properties for _ZN5vlstm7kernels8vlstm_fwI13__nv_bfloat16Li4ELi8ELi8EEEvPT_S4_S4_S4_iiii
#     24 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
# ptxas info    : Used 63 registers, 400 bytes cmem[0]

INCLUDE: ['/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/TH', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/THC', '/home/max/miniconda3/envs/xlstmpt220cu121/include']
/home/max/myrepos/vlstm_cuda/src
/home/max/cpplibs/libtorch/lib:/usr/local/cuda-12.3/lib64:
/home/max/miniconda3/envs/xlstmpt220cu121/lib


Using /home/max/.cache/torch_extensions/py311_cu121 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/max/.cache/torch_extensions/py311_cu121/vlstm_fw_v1/build.ninja...
Building extension module vlstm_fw_v1...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/2] /home/max/miniconda3/envs/xlstmpt220cu121/bin/nvcc --generate-dependencies-with-compile --dependency-output kernels.cuda.o.d -ccbin /home/max/miniconda3/envs/xlstmpt220cu121/bin/x86_64-conda-linux-gnu-cc -DTORCH_EXTENSION_NAME=vlstm_fw_v1 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/torch/csrc/api/include -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/TH -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/THC -isystem /home/max/miniconda3/envs/xlstmpt220cu121/include -isystem /home/max/miniconda3/envs/xlstmpt220cu121/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERSIONS

Loading extension module vlstm_fw_v1...


## CUDA vLSTM forward

In [3]:
S = 8 #16 #16 #8 # seq len
B = 1 # batch size
NH = 1 # num heads
DH = 8 # dim per head
DTYPE = torch.float32
DEVICE = torch.device("cuda:0")

In [4]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 10.
ks = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
vs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
igs = (1. + torch.arange((B * NH * S), device=DEVICE, dtype=DTYPE)).reshape(B, NH, S, 1) / 10.
# igs = torch.zeros((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
fgs = torch.ones((B, NH, S, 1), device=DEVICE, dtype=DTYPE)
qs, qs.shape, len(qs.view(-1))

(tensor([[[[0.0000, 0.1000, 0.2000, 0.3000, 0.4000, 0.5000, 0.6000, 0.7000],
           [0.8000, 0.9000, 1.0000, 1.1000, 1.2000, 1.3000, 1.4000, 1.5000],
           [1.6000, 1.7000, 1.8000, 1.9000, 2.0000, 2.1000, 2.2000, 2.3000],
           [2.4000, 2.5000, 2.6000, 2.7000, 2.8000, 2.9000, 3.0000, 3.1000],
           [3.2000, 3.3000, 3.4000, 3.5000, 3.6000, 3.7000, 3.8000, 3.9000],
           [4.0000, 4.1000, 4.2000, 4.3000, 4.4000, 4.5000, 4.6000, 4.7000],
           [4.8000, 4.9000, 5.0000, 5.1000, 5.2000, 5.3000, 5.4000, 5.5000],
           [5.6000, 5.7000, 5.8000, 5.9000, 6.0000, 6.1000, 6.2000, 6.3000]]]], device='cuda:0'),
 torch.Size([1, 1, 8, 8]),
 64)

In [5]:
# result_gt = vlstm_parallel_fw_torch(queries=qs, keys=ks, values=vs, igate_preact=igs, fgate_preact=fgs)
# result_gt, result_gt.shape

In [6]:
# pytorch version QKV product (To test if this is still correct after changes to the code.)
# at some point we have to compare to the vlstm_fw_torch version.
# rs = qs @ ks.transpose(-1, -2) @ vs
# rs, rs.shape

In [7]:
# rs_torch = (qs @ ks.transpose(-1, -2) * torch.tril(torch.ones((B, NH, S, S))).to(device=DEVICE, dtype=DTYPE)) @ vs
# rs_torch, rs_torch.shape

In [8]:
result_gt, m, l, n = vlstm_parallel_tiled(queries=qs, keys=ks, values=vs, igate_preact=igs, fgate_preact=fgs)
result_gt, result_gt.shape, m, l, n

q_tiles: 1, torch.Size([1, 1, 8, 8])
kv_tiles: 1, torch.Size([1, 1, 8, 8])


(tensor([[[[0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003],
           [0.0030, 0.0030, 0.0030, 0.0030, 0.0030, 0.0030, 0.0030, 0.0030],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100]]]], device='cuda:0'),
 torch.Size([1, 1, 8, 8]),
 tensor([[[[1.],
           [2.],
           [3.],
           [4.],
           [5.],
           [6.],
           [7.],
           [8.]]]], device='cuda:0'),
 tensor([[[[0.0099],
           [0.0413],
           [0.0740],
           [0.1058],
           [0.1372],
           [0.1682],
           [0.1992],
           [0.2302]]]], device='

In [9]:
# pytorch version
rs, log_fgm, igm, matD = vlstm_fw_torch(
    queries=qs,
    keys=ks,
    values=vs,
    igate_preact=igs,
    fgate_preact=fgs,
)
rs, rs.shape

(tensor([[[[0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003],
           [0.0030, 0.0030, 0.0030, 0.0030, 0.0030, 0.0030, 0.0030, 0.0030],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100]]]], device='cuda:0'),
 torch.Size([1, 1, 8, 8]))

In [10]:
# qTileIdx=0, kvTileIdx=0, cTileBlockXIdx=0, cTileBlockYIdx=0, hWTileSMIdxY=0, tbIdxXY=(0,0): m_val=1.000000, m_prev_val=-inf, n_val=0.367879, n_prev_val=0.000000
# qTileIdx=0, kvTileIdx=0, cTileBlockXIdx=0, cTileBlockYIdx=0, hWTileSMIdxY=1, tbIdxXY=(0,1): m_val=2.000000, m_prev_val=-inf, n_val=0.135335, n_prev_val=0.000000
# qTileIdx=0, kvTileIdx=0, cTileBlockXIdx=0, cTileBlockYIdx=0, hWTileSMIdxY=2, tbIdxXY=(0,2): m_val=3.000000, m_prev_val=-inf, n_val=0.209238, n_prev_val=0.000000
# qTileIdx=0, kvTileIdx=0, cTileBlockXIdx=0, cTileBlockYIdx=0, hWTileSMIdxY=3, tbIdxXY=(0,3): m_val=4.000000, m_prev_val=-inf, n_val=0.299359, n_prev_val=0.000000
# qTileIdx=0, kvTileIdx=0, cTileBlockXIdx=0, cTileBlockYIdx=0, hWTileSMIdxY=4, tbIdxXY=(0,0): m_val=5.000000, m_prev_val=-inf, n_val=0.387931, n_prev_val=0.000000
# qTileIdx=0, kvTileIdx=0, cTileBlockXIdx=0, cTileBlockYIdx=0, hWTileSMIdxY=5, tbIdxXY=(0,1): m_val=6.000000, m_prev_val=-inf, n_val=0.475842, n_prev_val=0.000000
# qTileIdx=0, kvTileIdx=0, cTileBlockXIdx=0, cTileBlockYIdx=0, hWTileSMIdxY=6, tbIdxXY=(0,2): m_val=7.000000, m_prev_val=-inf, n_val=0.563509, n_prev_val=0.000000
# qTileIdx=0, kvTileIdx=0, cTileBlockXIdx=0, cTileBlockYIdx=0, hWTileSMIdxY=7, tbIdxXY=(0,3): m_val=8.000000, m_prev_val=-inf, n_val=0.651093, n_prev_val=0.000000

In [11]:
# cuda kernel
rs_cuda, matC = vlstm_fw_cuda(mat_Q=qs, mat_K=ks, mat_V=vs, igate_preact=igs.squeeze(-1), fgate_preact=fgs.squeeze(-1))
rs_cuda, rs_cuda.shape

before kernel dispatch - float32!
B: 1, NH: 1, S: 8, DH: 8
blocksxy: 1-1, threadsxy: 4-4, shared_mem in bytes: 5700
In Kernel: gdim.x: 1, gdim.y: 1, gdim.z: 1, bdim.x: 4, bdim.y: 4
In Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4
qTileIdx=0, kvTileIdx=0, cTileBlockXIdx=0, cTileBlockYIdx=0, lThreadSMIdx=0, tbIdxXY=(0,0): l_val=0.028000, exp(-m_val)=0.367879, n_val=0.367879
qTileIdx=0, kvTileIdx=0, cTileBlockXIdx=0, cTileBlockYIdx=0, lThreadSMIdx=1, tbIdxXY=(1,0): l_val=0.116743, exp(-m_val)=0.135335, n_val=0.135335
qTileIdx=0, kvTileIdx=0, cTileBlockXIdx=0, cTileBlockYIdx=0, lThreadSMIdx=2, tbIdxXY=(2,0): l_val=0.209238, exp(-m_val)=0.049787, n_val=0.209238
qTileIdx=0, kvTileIdx=0, cTileBlockXIdx=0, cTileBlockYIdx=0, lThreadSMIdx=3, tbIdxXY=(3,0): l_val=0.299359, exp(-m_val)=0.018316, n_val=0.299359
qTileIdx=0, kvTileIdx=0, cTileBlockXIdx=0, cTileBlockYIdx=0, lThreadSMIdx=4, tbIdxXY=(0,1): l_val=0.387931, exp(-m_val)=0.006738, n_val=0.387931
qTileIdx=0, kvTileIdx=0, cTileBlockXIdx=0, c

(tensor([[[[0.0008, 0.0008, 0.0008, 0.0008, 0.0008, 0.0008, 0.0008, 0.0008],
           [0.0086, 0.0086, 0.0086, 0.0086, 0.0086, 0.0086, 0.0086, 0.0086],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100]]]], device='cuda:0'),
 torch.Size([1, 1, 8, 8]))

In [12]:
rs - rs_cuda

tensor([[[[-4.9202e-04, -4.9202e-04, -4.9202e-04, -4.9202e-04, -4.9202e-04, -4.9202e-04, -4.9202e-04, -4.9202e-04],
          [-5.5764e-03, -5.5764e-03, -5.5764e-03, -5.5764e-03, -5.5764e-03, -5.5764e-03, -5.5764e-03, -5.5764e-03],
          [-1.3411e-07, -1.3411e-07, -1.3411e-07, -1.3411e-07, -1.3411e-07, -1.3411e-07, -1.3411e-07, -1.3411e-07],
          [-9.4064e-08, -9.4064e-08, -9.4064e-08, -9.4064e-08, -9.4064e-08, -9.4064e-08, -9.4064e-08, -9.4064e-08],
          [-7.3574e-08, -7.3574e-08, -7.3574e-08, -7.3574e-08, -7.3574e-08, -7.3574e-08, -7.3574e-08, -7.3574e-08],
          [-5.9605e-08, -5.9605e-08, -5.9605e-08, -5.9605e-08, -5.9605e-08, -5.9605e-08, -5.9605e-08, -5.9605e-08],
          [-5.0291e-08, -5.0291e-08, -5.0291e-08, -5.0291e-08, -5.0291e-08, -5.0291e-08, -5.0291e-08, -5.0291e-08],
          [-4.3772e-08, -4.3772e-08, -4.3772e-08, -4.3772e-08, -4.3772e-08, -4.3772e-08, -4.3772e-08, -4.3772e-08]]]], device='cuda:0')

In [13]:
matC.shape

torch.Size([1, 1, 8, 8])

In [14]:
matC

tensor([[[[ 1.0000,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
          [ 0.6867,  2.0000,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
          [ 0.3735,  1.6867,  3.0000,    -inf,    -inf,    -inf,    -inf,    -inf],
          [ 0.0602,  1.3735,  2.6867,  4.0000,    -inf,    -inf,    -inf,    -inf],
          [-0.2530,  1.0602,  2.3735,  3.6867,  5.0000,    -inf,    -inf,    -inf],
          [-0.5663,  0.7470,  2.0602,  3.3735,  4.6867,  6.0000,    -inf,    -inf],
          [-0.8796,  0.4337,  1.7470,  3.0602,  4.3735,  5.6867,  7.0000,    -inf],
          [-1.1928,  0.1204,  1.4337,  2.7470,  4.0602,  5.3735,  6.6867,  8.0000]]]], device='cuda:0')

In [15]:
matD

tensor([[[[ 1.0000,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
          [ 0.6867,  2.0000,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
          [ 0.3735,  1.6867,  3.0000,    -inf,    -inf,    -inf,    -inf,    -inf],
          [ 0.0602,  1.3735,  2.6867,  4.0000,    -inf,    -inf,    -inf,    -inf],
          [-0.2530,  1.0602,  2.3735,  3.6867,  5.0000,    -inf,    -inf,    -inf],
          [-0.5663,  0.7470,  2.0602,  3.3735,  4.6867,  6.0000,    -inf,    -inf],
          [-0.8796,  0.4337,  1.7470,  3.0602,  4.3735,  5.6867,  7.0000,    -inf],
          [-1.1928,  0.1204,  1.4337,  2.7470,  4.0602,  5.3735,  6.6867,  8.0000]]]], device='cuda:0')

In [16]:
matC - matD

tensor([[[[0.0000e+00,        nan,        nan,        nan,        nan,        nan,        nan,        nan],
          [5.9605e-08, 0.0000e+00,        nan,        nan,        nan,        nan,        nan,        nan],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,        nan,        nan,        nan,        nan,        nan],
          [5.9605e-08, 1.1921e-07, 0.0000e+00, 0.0000e+00,        nan,        nan,        nan,        nan],
          [1.1921e-07, 2.3842e-07, 0.0000e+00, 0.0000e+00, 0.0000e+00,        nan,        nan,        nan],
          [1.1921e-07, 2.3842e-07, 2.3842e-07, 0.0000e+00, 4.7684e-07, 0.0000e+00,        nan,        nan],
          [0.0000e+00, 0.0000e+00, 1.1921e-07, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,        nan],
          [0.0000e+00, 0.0000e+00, 1.1921e-07, 2.3842e-07, 0.0000e+00, 0.0000e+00, 4.7684e-07, 0.0000e+00]]]], device='cuda:0')